<a href="https://colab.research.google.com/github/anyaagarwal5/TEXT-SUMMARIZATION/blob/main/abstractive_mdel_t5_model_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split

import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

data = pd.read_csv('/content/drive/MyDrive/datset 1/preprocessed.csv')

In [ ]:
train_val  , test_data  =  train_test_split(data , test_size=0.2 , random_state= 42 )

train_data , validation_data = train_test_split(train_val , test_size= 0.1 , random_state= 42)

In [ ]:
from operator import index

#saving the data to csv files

test_data.to_csv('test.csv', index= False)

train_data.to_csv('train.csv' , index= False)

validation_data.to_csv('val.csv' , index = False)

In [ ]:
train_val.describe()

,text,summary
count,3516,3516
unique,3477,3516
top,aaj tak live tv with live election results i c...,"criticising the up chief minister, bjp mp yogi..."
freq,9,1


In [ ]:
data

,text,summary
0,the daman and diu administration on wednesday ...,the administration of union territory daman an...
1,"from her special numbers to tv?appearances, bo...",malaika arora slammed an instagram user who tr...
2,the indira gandhi institute of medical science...,the indira gandhi institute of medical science...
3,lashkar-e-taiba's kashmir commander abu dujana...,lashkar-e-taiba's kashmir commander abu dujana...
4,hotels in mumbai and other indian cities are t...,hotels in maharashtra will train their staff t...
...,...,...
4391,"mumbai, feb (pti) fruit juice concentrate m...",fruit juice concentrate maker rasna is eyeing ...
4392,former cricketer sachin tendulkar was spotted ...,former indian cricketer sachin tendulkar atten...
4393,"aamir khan, whose last film dangal told the st...","aamir khan, while talking about reality shows ..."
4394,maharahstra power minister chandrashekhar bawa...,the maharashtra government has initiated an in...


In [ ]:
#using the pretrained t5-small model
from tqdm import tqdm

import torch

from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
model_name = 't5-small'

#loading the tokenizer and model

tokenizer = T5Tokenizer.from_pretrained(model_name)

model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def summarize(text, model, tokenizer, max_length=150, min_length=40, length_penalty=2.0, num_beams=3):

    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    #inputs = inputs.to(device)

    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
import pandas as pd
test =  pd.read_csv('test.csv')

In [ ]:

test = test.rename(columns={"summary":"generated_text", "text":"input_text"})
test['prefix'] = ''
test

,input_text,target_text,prefix
0,"washington, mar (pti) a -year-old woman i...","a woman, who was arrested twice last week for ...",
1,the tea stall in guajarat's vadnagar where pri...,a tea stall in gujarat's vadnagar railway stat...,
2,"continuing a yearly tradition, outgoing us pre...",us president barack obama has declared january...,
3,the supreme court on thursday rejected a petit...,the supreme court on thursday rejected a petit...,
4,"captain amarinder singh, the congress's chief ...",congress leader captain amarinder singh will t...,
...,...,...,...
875,"new delhi, dec (pti) dismissing the petition...",while dismissing a petition against demonetisa...,
876,"there's strange, there's unfathomable, then th...",a dutch game show was slammed by people for an...,
877,taimur ali khan is undoubtedly one of the cute...,pictures of saif ali khan and kareena kapoor k...,
878,a preliminary enquiry has been ordered by the...,the economic offences wing of the delhi police...,


In [ ]:
# using sumeval to calculate ROUGE scores , faster way to calculate ROUGE scores
!pip install sumeval
from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.0 MB/s eta 0:00:00
  Created wheel for sumeval: filename=sumeval-0.2.2-py3-none-any.whl size=54530 sha256=ad668b176fb839a4d3b5cb56016753dbbe26085473c2013ecc2577705ecd4670
  Stored in directory: /root/.cache/pip/wheels/94/08/ae/a1150487990be9a1f8530616bde7bbb288dfecd1fc75a8c930
Successfully built sumeval


In [ ]:
import numpy as np
def rouge_calc(preds, targets):
    rouge_1 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=1) for i in range(len(preds))]
    rouge_2 = [rouge.rouge_n(summary=preds[i],references=targets[i],n=2) for i in range(len(preds))]
    rouge_l = [rouge.rouge_l(summary=preds[i],references=targets[i]) for i in range(len(preds))]

    return {"Rouge_1": np.array(rouge_1).mean(),
            "Rouge_2": np.array(rouge_2).mean(),
            "Rouge_L": np.array(rouge_l).mean()}

In [ ]:
test = test.rename(columns={"summary":"target_text", "text":"input_text"})  # Correcting the typo 'coulun' to 'column'
# Now you can calculate the ROUGE scores
rouge_calc(list(test['target_text']), list(test['input_text']))

{'Rouge_1': 0.29047139736445204,
 'Rouge_2': 0.16261992912218956,
 'Rouge_L': 0.2283350641827908}